In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("whitegrid")
from pylab import figure, axes, pie, title, savefig

In [2]:
JR_2013 = pd.read_csv('NHIS_OPEN_T20_2013.csv', encoding='CP949')
JR_2014 = pd.read_csv('NHIS_OPEN_T20_2014.csv', encoding='CP949')
JR_2015 = pd.read_csv('NHIS_OPEN_T20_2015.csv', encoding='CP949')

In [3]:
JR_2010_2015 = pd.concat([JR_2013, JR_2014, JR_2015])
JR_2010_2015.head()

,기준년도,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,데이터 기준일자
0,2013,287350,1,2,9,11,20131212,3,14,L033,R195,1,1,15,13190,3900,9290,4,20151220
1,2013,96615,2,1,11,11,20131227,3,14,L259,NaN,1,1,15,13190,3900,9290,4,20151220
2,2013,96615,3,1,11,11,20131231,3,14,L259,NaN,1,1,15,9430,2800,6630,4,20151220
3,2013,295197,4,2,4,11,20131227,3,14,L259,NaN,1,1,15,13190,3900,9290,1,20151220
4,2013,586622,5,1,16,11,20131210,3,14,N408,NaN,1,1,15,9430,1500,7930,30,20151220


## Counting Patient with treat_kind 1 (Month Criteria)

In [4]:
tk_2010_2015 = JR_2010_2015[['요양개시일자', '진료과목코드']]
tk_2010_2015['요양개시일자'] = pd.to_datetime(tk_2010_2015['요양개시일자'], format = '%Y%m%d', errors = 'coerce')
tk_2010_2015_code1 = tk_2010_2015[tk_2010_2015['진료과목코드'] == 1]
tk_2010_2015_code1_ct = tk_2010_2015_code1['요양개시일자'].groupby(tk_2010_2015_code1.요양개시일자.dt.month).agg('count')
tk_2010_2015_code1_ct = pd.DataFrame(tk_2010_2015_code1_ct)
plt.figure()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns= {'요양개시일자': 'Total_patients'})
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.reset_index()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns = {'요양개시일자': 'Month'})
tk_2010_2015_code1_ct.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Month,Total_patients
0,1,905777
1,2,852628
2,3,939279
3,4,923154
4,5,852992


## Visualization treat_kind 1 with Month

In [5]:
f, ax = plt.subplots(figsize = (10, 15))

# plot the total treat_kind 1
sns.barplot(x = 'Month', y = 'Total_patients', data = tk_2010_2015_code1_ct)
plt.savefig('내과 월별 이용 인원.png', dpi=1000)

## Find max value and making new table (3 large value)

In [6]:
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.sort_values(by='Total_patients', ascending = False)
tk_2010_2015_code1_ct_large = tk_2010_2015_code1_ct.iloc[0:3]
tk_2010_2015_code1_ct_large

,Month,Total_patients
11,12,964808
2,3,939279
3,4,923154


In [7]:
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct.iloc[3:12]
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct_ect['Total_patients'].sum()
tk_2010_2015_code1_ct_large.loc[4] = ['ect', tk_2010_2015_code1_ct_ect]
tk_2010_2015_code1_ct_graph = tk_2010_2015_code1_ct_large.reset_index().drop('index', axis = 1)
tk_2010_2015_code1_ct_graph

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Month,Total_patients
0,12,964808
1,3,939279
2,4,923154
3,ect,7462274


In [8]:
color = ['forestgreen', 'limegreen', 'palegreen', 'honeydew']
tk_2010_2015_code1_ct_graph.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 90, labels = tk_2010_2015_code1_ct_graph['Month'], legend = True, colors = color)
# plt.pie(tk_2010_2015_code1_ct_graph['Total_patients'], shadow = True, startangle = 0, autopct='%1.1f%%')
plt.axis('equal')
plt.tight_layout()
savefig('내과 월별 이용 인원 pie chart.png', dpi = 1000)


## Find 3 large main_sick with Month

In [9]:
JR_2010_2015 = JR_2010_2015[['요양개시일자', '진료과목코드', '주상병코드']].rename(columns={'요양개시일자': 'rec_date', '진료과목코드': 'treat_kind', '주상병코드': 'main_sick'})
ms_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind', 'main_sick']]
ms_2010_2015['rec_date'] = pd.to_datetime(ms_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
ms_2010_2015_code1 = ms_2010_2015[ms_2010_2015['treat_kind'] == 1]
ms_2010_2015_code1_st = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[0, 'Month']
ms_2010_2015_code1_st = ms_2010_2015_code1[ms_2010_2015_code1_st]
ms_2010_2015_code1_st.head()

,rec_date,treat_kind,main_sick
33,2013-12-05,1,J069
40,2013-12-16,1,J458
41,2013-12-21,1,J303
42,2013-12-26,1,J460
70,2013-12-09,1,J450


In [10]:
ms_2010_2015_code1_nd = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[1, 'Month']
ms_2010_2015_code1_nd = ms_2010_2015_code1[ms_2010_2015_code1_nd]
ms_2010_2015_code1_nd.head()

,rec_date,treat_kind,main_sick
32074,2013-03-11,1,J459
34237,2013-03-04,1,I841
34238,2013-03-12,1,I843
36056,2013-03-30,1,A049
36060,2013-03-05,1,B07


In [11]:
ms_2010_2015_code1_th = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[2, 'Month']
ms_2010_2015_code1_th = ms_2010_2015_code1[ms_2010_2015_code1_th]
ms_2010_2015_code1_th.head()

,rec_date,treat_kind,main_sick
46857,2013-04-04,1,I841
49360,2013-04-18,1,A160
49361,2013-04-20,1,K769
49362,2013-04-23,1,A168
49363,2013-04-26,1,J998


## First Month Find 3 Large main_sick

In [12]:
a1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [13]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    8531
 Name: A090, dtype: int64, counts    5136
 Name: B181, dtype: int64, counts    2260
 Name: C73, dtype: int64, counts    1758
 Name: D509, dtype: int64, counts    35246
 Name: E119, dtype: int64, counts    555
 Name: F009, dtype: int64, counts    735
 Name: G470, dtype: int64, counts    242
 Name: H814, dtype: int64, counts    117976
 Name: I109, dtype: int64, counts    107736
 Name: J209, dtype: int64, counts    25728
 Name: K210, dtype: int64, counts    971
 Name: L239, dtype: int64, counts    3240
 Name: M545, dtype: int64, counts    16792
 Name: N185, dtype: int64, counts    204
 Name: O244, dtype: int64, counts    4
 Name: P599, dtype: int64, counts    78
 Name: Q612, dtype: int64, counts    3363
 Name: R074, dtype: int64, counts    928
 Name: S335, dtype: int64, counts    566
 Name: T232, dtype: int64, counts    71
 Name: U880, dtype: int64, counts    586
 Name: Z961, dtype: int64]

In [31]:
code1_st_graph1 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('I109')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'I109': 'Unspecified hypertension'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J209')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('E119')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'E119': 'Non-insulin-dependent diabetes mellitus'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot.pie(y='Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'December', colors = color, figsize = (15, 10) )
plt.axis('equal')
plt.tight_layout()
plt.savefig('내과 12월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Second Month Find 3 Large main_sick

In [32]:
a1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [33]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    6576
 Name: A090, dtype: int64, counts    4739
 Name: B181, dtype: int64, counts    2151
 Name: C73, dtype: int64, counts    1529
 Name: D509, dtype: int64, counts    32910
 Name: E119, dtype: int64, counts    465
 Name: F009, dtype: int64, counts    668
 Name: G470, dtype: int64, counts    240
 Name: H814, dtype: int64, counts    111830
 Name: I109, dtype: int64, counts    115206
 Name: J209, dtype: int64, counts    19296
 Name: K210, dtype: int64, counts    1147
 Name: L239, dtype: int64, counts    3375
 Name: M545, dtype: int64, counts    15520
 Name: N185, dtype: int64, counts    167
 Name: O244, dtype: int64, counts    1
 Name: P783, dtype: int64, counts    2151
 Name: C73, dtype: int64, counts    3238
 Name: R074, dtype: int64, counts    847
 Name: S335, dtype: int64, counts    502
 Name: T232, dtype: int64, counts    88
 Name: U880, dtype: int64, counts    637
 Name: Z961, dtype: int64]

In [37]:
code1_st_graph1 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I109')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'I109': 'Unspecified hypertension'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('E119')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'E119': 'Non-insulin-dependent diabetes mellitus'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'March', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
plt.savefig('내과 3월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Third Month Find 3 Large main_sick¶

In [38]:
a1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [39]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
#code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    6346
 Name: A090, dtype: int64, counts    4839
 Name: B181, dtype: int64, counts    2161
 Name: C73, dtype: int64, counts    1556
 Name: D509, dtype: int64, counts    32798
 Name: E119, dtype: int64, counts    492
 Name: F009, dtype: int64, counts    699
 Name: G470, dtype: int64, counts    236
 Name: H814, dtype: int64, counts    111728
 Name: I109, dtype: int64, counts    108493
 Name: J209, dtype: int64, counts    19261
 Name: K210, dtype: int64, counts    1186
 Name: L239, dtype: int64, counts    3358
 Name: M545, dtype: int64, counts    15645
 Name: N185, dtype: int64, counts    173
 Name: O244, dtype: int64, counts    71
 Name: Q612, dtype: int64, counts    3061
 Name: R074, dtype: int64, counts    944
 Name: S335, dtype: int64, counts    602
 Name: T232, dtype: int64, counts    82
 Name: U880, dtype: int64, counts    615
 Name: Z961, dtype: int64]

In [40]:
code1_st_graph1 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I109')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'I109': 'Unspecified hypertension'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J209')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E119')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'E119': 'Non-insulin-dependent diabetes mellitus'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'April', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
plt.savefig('내과 4월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Visualization treat_kind 4 with Month

In [41]:
tk_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind']]
tk_2010_2015['rec_date'] = pd.to_datetime(tk_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
tk_2010_2015_code1 = tk_2010_2015[tk_2010_2015['treat_kind'] == 5]
tk_2010_2015_code1_ct = tk_2010_2015_code1['rec_date'].groupby(tk_2010_2015_code1.rec_date.dt.month).agg('count')
tk_2010_2015_code1_ct = pd.DataFrame(tk_2010_2015_code1_ct)
plt.figure()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns= {'rec_date': 'Total_patients'})
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.reset_index()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns = {'rec_date': 'Month'})
tk_2010_2015_code1_ct

f, ax = plt.subplots(figsize = (10, 15))

# plot the total treat_kind 4
sns.barplot(x = 'Month', y = 'Total_patients', data = tk_2010_2015_code1_ct)
plt.savefig('정형외과 월별 이용 인원.png', dpi=1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find max value and making new table (3 large value)

In [42]:
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.sort_values(by='Total_patients', ascending = False)
tk_2010_2015_code1_ct_large = tk_2010_2015_code1_ct.iloc[0:3]
tk_2010_2015_code1_ct_large

,Month,Total_patients
6,7,533476
3,4,510679
4,5,506484


In [43]:
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct.iloc[3:12]
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct_ect['Total_patients'].sum()
tk_2010_2015_code1_ct_large.loc[1] = ['ect', tk_2010_2015_code1_ct_ect]
tk_2010_2015_code1_ct_graph = tk_2010_2015_code1_ct_large.reset_index().drop('index', axis = 1)
tk_2010_2015_code1_ct_graph

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Month,Total_patients
0,7,533476
1,4,510679
2,5,506484
3,ect,4314180


In [44]:
color = ['forestgreen', 'limegreen', 'palegreen', 'honeydew']
tk_2010_2015_code1_ct_graph.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 90, labels = tk_2010_2015_code1_ct_graph['Month'], legend = True, colors = color)
# plt.pie(tk_2010_2015_code1_ct_graph['Total_patients'], shadow = True, startangle = 0, autopct='%1.1f%%')
plt.axis('equal')
plt.tight_layout()
savefig('정형외과 월별 이용 인원 pie chart.png', dpi=1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [45]:
ms_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind', 'main_sick']]
ms_2010_2015['rec_date'] = pd.to_datetime(ms_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
ms_2010_2015_code1 = ms_2010_2015[ms_2010_2015['treat_kind'] == 5]
ms_2010_2015_code1_st = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[0, 'Month']
ms_2010_2015_code1_st = ms_2010_2015_code1[ms_2010_2015_code1_st]
ms_2010_2015_code1_st.head()

,rec_date,treat_kind,main_sick
6071,2013-07-17,5,S936
20092,2013-07-26,5,M138
20094,2013-07-22,5,M791
20095,2013-07-06,5,M190
20096,2013-07-09,5,S934


In [46]:
ms_2010_2015_code1_nd = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[1, 'Month']
ms_2010_2015_code1_nd = ms_2010_2015_code1[ms_2010_2015_code1_nd]
ms_2010_2015_code1_nd.head()

,rec_date,treat_kind,main_sick
49483,2013-04-09,5,S134
49504,2013-04-05,5,S335
49560,2013-04-04,5,M796
49577,2013-04-26,5,M139
49584,2013-04-03,5,M139


In [47]:
ms_2010_2015_code1_th = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[2, 'Month']
ms_2010_2015_code1_th = ms_2010_2015_code1[ms_2010_2015_code1_th]
ms_2010_2015_code1_th.head()

,rec_date,treat_kind,main_sick
63732,2013-05-14,5,M796
63733,2013-05-18,5,S015
63750,2013-05-15,5,S013
63752,2013-05-25,5,M139
63755,2013-05-16,5,M139


## First Month Find 3 Large main_sick

In [48]:
a1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [49]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    43
 Name: A099, dtype: int64, counts    241
 Name: B351, dtype: int64, counts    35
 Name: C402, dtype: int64, counts    154
 Name: D172, dtype: int64, counts    187
 Name: E119, dtype: int64, counts    77
 Name: F009, dtype: int64, counts    894
 Name: G560, dtype: int64, counts    15
 Name: H000, dtype: int64, counts    1008
 Name: I109, dtype: int64, counts    317
 Name: J209, dtype: int64, counts    172
 Name: K297, dtype: int64, counts    1138
 Name: L600, dtype: int64, counts    32118
 Name: M170, dtype: int64, counts    44
 Name: N309, dtype: int64, counts    2
 Name: O989, dtype: int64, counts    2
 Name: P220, dtype: int64, counts    24
 Name: Q668, dtype: int64, counts    160
 Name: R529, dtype: int64, counts    16465
 Name: S335, dtype: int64, counts    167
 Name: T141, dtype: int64, counts    176
 Name: Z988, dtype: int64]

In [50]:
code1_st_graph1 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('M170')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'M170': 'Primary gonarthrosis, bilateral'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('S335')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'S335': 'Sprain and strain of lumbar spine'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L600')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'L600': 'Ingrowing nail'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'July', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
plt.savefig('정형외과 7월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Second Month Find 3 Large main_sick

In [51]:
a1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [52]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    54
 Name: A099, dtype: int64, counts    189
 Name: B351, dtype: int64, counts    44
 Name: C402, dtype: int64, counts    125
 Name: D211, dtype: int64, counts    208
 Name: E119, dtype: int64, counts    48
 Name: F009, dtype: int64, counts    793
 Name: G560, dtype: int64, counts    10
 Name: H109, dtype: int64, counts    1006
 Name: I109, dtype: int64, counts    753
 Name: J209, dtype: int64, counts    210
 Name: K297, dtype: int64, counts    829
 Name: L600, dtype: int64, counts    32449
 Name: M170, dtype: int64, counts    46
 Name: N309, dtype: int64, counts    2
 Name: O800, dtype: int64, counts    2
 Name: P599, dtype: int64, counts    20
 Name: Q668, dtype: int64, counts    179
 Name: R529, dtype: int64, counts    14808
 Name: S335, dtype: int64, counts    158
 Name: T232, dtype: int64, counts    182
 Name: Z988, dtype: int64]

In [54]:
code1_st_graph1 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('M170')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'M170': 'Primary gonarthrosis, bilateral'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('S335')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'S335': 'Sprain and strain of lumbar spine'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I109')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'I109': 'Unspecified hypertension'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'January', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
plt.savefig('정형외과 4월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Third Month Find 3 Large main_sick¶

In [55]:
a1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [56]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    52
 Name: A099, dtype: int64, counts    208
 Name: B351, dtype: int64, counts    36
 Name: C402, dtype: int64, counts    168
 Name: D172, dtype: int64, counts    187
 Name: E119, dtype: int64, counts    56
 Name: F009, dtype: int64, counts    705
 Name: G560, dtype: int64, counts    14
 Name: H814, dtype: int64, counts    1041
 Name: I109, dtype: int64, counts    654
 Name: J209, dtype: int64, counts    145
 Name: K297, dtype: int64, counts    875
 Name: L031, dtype: int64, counts    32591
 Name: M170, dtype: int64, counts    55
 Name: N400, dtype: int64, counts    1
 Name: O912, dtype: int64, counts    4
 Name: P38, dtype: int64, counts    25
 Name: Q668, dtype: int64, counts    149
 Name: R529, dtype: int64, counts    14603
 Name: S335, dtype: int64, counts    159
 Name: T232, dtype: int64, counts    174
 Name: Z988, dtype: int64]

In [57]:
code1_st_graph1 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('M170')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'M170': 'Primary gonarthrosis, bilateral'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('S335')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'S335': 'Sprain and strain of lumbar spine'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I109')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'I109': 'Unspecified hypertension'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'October', colors = color, figsize= (15, 10))
plt.axis('equal')
plt.tight_layout()
plt.savefig('정형외과 5월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Counting Patient with treat_kind 11 (Month Criteria)

In [58]:
tk_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind']]
tk_2010_2015['rec_date'] = pd.to_datetime(tk_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
tk_2010_2015_code1 = tk_2010_2015[tk_2010_2015['treat_kind'] == 11]
tk_2010_2015_code1_ct = tk_2010_2015_code1['rec_date'].groupby(tk_2010_2015_code1.rec_date.dt.month).agg('count')
tk_2010_2015_code1_ct = pd.DataFrame(tk_2010_2015_code1_ct)
plt.figure()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns= {'rec_date': 'Total_patients'})
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.reset_index()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns = {'rec_date': 'Month'})
tk_2010_2015_code1_ct

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


,Month,Total_patients
0,1,272941
1,2,294245
2,3,281274
3,4,293314
4,5,263937
5,6,218931
6,7,218969
7,8,211299
8,9,258713
9,10,272891


## Visualization treat_kind 11 with Month

In [59]:
f, ax = plt.subplots(figsize = (10, 15))

# plot the total treat_kind 11
sns.barplot(x = 'Month', y = 'Total_patients', data = tk_2010_2015_code1_ct)
plt.savefig('소아청소년과 월별 이용 인원.png', dpi=1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find max value and making new table (3 large value)

In [60]:
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.sort_values(by='Total_patients', ascending = False)
tk_2010_2015_code1_ct_large = tk_2010_2015_code1_ct.iloc[0:3]
tk_2010_2015_code1_ct_large

,Month,Total_patients
1,2,294245
3,4,293314
11,12,288704


In [61]:
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct.iloc[3:12]
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct_ect['Total_patients'].sum()
tk_2010_2015_code1_ct_large.loc[4] = ['ect', tk_2010_2015_code1_ct_ect]
tk_2010_2015_code1_ct_graph = tk_2010_2015_code1_ct_large.reset_index().drop('index', axis = 1)
tk_2010_2015_code1_ct_graph

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Month,Total_patients
0,2,294245
1,4,293314
2,12,288704
3,ect,2282957


In [62]:
color = ['forestgreen', 'limegreen', 'palegreen', 'honeydew']
tk_2010_2015_code1_ct_graph.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 90, labels = tk_2010_2015_code1_ct_graph['Month'], legend = True, colors = color)
# plt.pie(tk_2010_2015_code1_ct_graph['Total_patients'], shadow = True, startangle = 0, autopct='%1.1f%%')
plt.axis('equal')
plt.tight_layout()
savefig('소아청소년과 월별 이용 인원 pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find 3 large main_sick with Month

In [63]:
ms_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind', 'main_sick']]
ms_2010_2015['rec_date'] = pd.to_datetime(ms_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
ms_2010_2015_code1 = ms_2010_2015[ms_2010_2015['treat_kind'] == 11]
ms_2010_2015_code1_st = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[0, 'Month']
ms_2010_2015_code1_st = ms_2010_2015_code1[ms_2010_2015_code1_st]
ms_2010_2015_code1_st.head()

,rec_date,treat_kind,main_sick
23323,2013-02-23,11,K297
23487,2013-02-12,11,J019
23488,2013-02-05,11,J00
23489,2013-02-07,11,J209
23490,2013-02-01,11,J039


In [64]:
ms_2010_2015_code1_nd = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[1, 'Month']
ms_2010_2015_code1_nd = ms_2010_2015_code1[ms_2010_2015_code1_nd]
ms_2010_2015_code1_nd.head()

,rec_date,treat_kind,main_sick
47834,2013-04-10,11,J353
49320,2013-04-25,11,H259
49370,2013-04-25,11,B370
49371,2013-04-29,11,B370
49372,2013-04-08,11,A090


In [65]:
ms_2010_2015_code1_th = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[2, 'Month']
ms_2010_2015_code1_th = ms_2010_2015_code1[ms_2010_2015_code1_th]
ms_2010_2015_code1_th.head()

,rec_date,treat_kind,main_sick
39,2013-12-03,11,B374
149,2013-12-07,11,J018
182,2013-12-27,11,J010
185,2013-12-19,11,J209
191,2013-12-31,11,J010


## First Month Find 3 Large main_sick

In [66]:
a1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [67]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    4900
 Name: A090, dtype: int64, counts    378
 Name: B07, dtype: int64, counts    134
 Name: C910, dtype: int64, counts    237
 Name: D509, dtype: int64, counts    1503
 Name: E301, dtype: int64, counts    14
 Name: F009, dtype: int64, counts    302
 Name: G409, dtype: int64, counts    2283
 Name: H660, dtype: int64, counts    1174
 Name: I109, dtype: int64, counts    87521
 Name: J209, dtype: int64, counts    1156
 Name: K529, dtype: int64, counts    833
 Name: L209, dtype: int64, counts    398
 Name: M791, dtype: int64, counts    267
 Name: N390, dtype: int64, counts    3
 Name: O249, dtype: int64, counts    687
 Name: P599, dtype: int64, counts    97
 Name: Q381, dtype: int64, counts    1781
 Name: R509, dtype: int64, counts    75
 Name: S335, dtype: int64, counts    230
 Name: T232, dtype: int64, counts    1107
 Name: Z380, dtype: int64]

In [69]:
code1_st_graph1 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H660')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'H660': 'Acute suppurative otitis media'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('R509')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'R509': 'Fever, unspecified'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'February', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('소아청소년과 2월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Second Month Find 3 Large main_sick

In [70]:
a1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [71]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    4048
 Name: A090, dtype: int64, counts    800
 Name: B084, dtype: int64, counts    187
 Name: C910, dtype: int64, counts    261
 Name: D509, dtype: int64, counts    1443
 Name: E301, dtype: int64, counts    18
 Name: F980, dtype: int64, counts    260
 Name: G409, dtype: int64, counts    3840
 Name: H669, dtype: int64, counts    1391
 Name: I109, dtype: int64, counts    88323
 Name: J209, dtype: int64, counts    907
 Name: K529, dtype: int64, counts    920
 Name: L209, dtype: int64, counts    345
 Name: M791, dtype: int64, counts    323
 Name: N390, dtype: int64, counts    3
 Name: O924, dtype: int64, counts    727
 Name: P599, dtype: int64, counts    97
 Name: Q381, dtype: int64, counts    1671
 Name: R509, dtype: int64, counts    95
 Name: S335, dtype: int64, counts    154
 Name: T232, dtype: int64, counts    1176
 Name: Z380, dtype: int64]

In [72]:
code1_st_graph1 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H669')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'H669': 'Otitis media, unspecified'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('R509')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'R509': 'Fever, unspecified'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 90, labels = c1_st_123['main_sick'], legend = True, title = 'April', colors = color)
plt.axis('equal')
plt.tight_layout()
savefig('소아청소년과 4월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Third Month Find 3 Large main_sick¶

In [73]:
a1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [74]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    6742
 Name: A090, dtype: int64, counts    627
 Name: B019, dtype: int64, counts    201
 Name: C910, dtype: int64, counts    207
 Name: D509, dtype: int64, counts    1504
 Name: E301, dtype: int64, counts    23
 Name: F510, dtype: int64, counts    267
 Name: G409, dtype: int64, counts    3509
 Name: H660, dtype: int64, counts    1365
 Name: I109, dtype: int64, counts    86212
 Name: J209, dtype: int64, counts    1354
 Name: K529, dtype: int64, counts    691
 Name: L209, dtype: int64, counts    335
 Name: M791, dtype: int64, counts    277
 Name: N390, dtype: int64, counts    4
 Name: O912, dtype: int64, counts    555
 Name: P599, dtype: int64, counts    100
 Name: Q211, dtype: int64, counts    1445
 Name: R509, dtype: int64, counts    95
 Name: S335, dtype: int64, counts    154
 Name: T232, dtype: int64, counts    715
 Name: Z380, dtype: int64]

In [76]:
code1_st_graph1 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H660')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'H660': 'Acute suppurative otitis media'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E301')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'E301': 'Precocious puberty'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'December', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('소아청소년과 12월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Counting Patient with treat_kind 11 (Month Criteria)

In [77]:
tk_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind']]
tk_2010_2015['rec_date'] = pd.to_datetime(tk_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
tk_2010_2015_code1 = tk_2010_2015[tk_2010_2015['treat_kind'] == 13]
tk_2010_2015_code1_ct = tk_2010_2015_code1['rec_date'].groupby(tk_2010_2015_code1.rec_date.dt.month).agg('count')
tk_2010_2015_code1_ct = pd.DataFrame(tk_2010_2015_code1_ct)
plt.figure()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns= {'rec_date': 'Total_patients'})
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.reset_index()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns = {'rec_date': 'Month'})
tk_2010_2015_code1_ct

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


,Month,Total_patients
0,1,239181
1,2,236156
2,3,255965
3,4,266187
4,5,232703
5,6,175977
6,7,169907
7,8,175824
8,9,236621
9,10,247687


## Visualization treat_kind 11 with Month

In [78]:
f, ax = plt.subplots(figsize = (10, 15))

# plot the total treat_kind 11
sns.barplot(x = 'Month', y = 'Total_patients', data = tk_2010_2015_code1_ct)
plt.savefig('이비인후과 월별 이용 인원.png', dpi=1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find max value and making new table (3 large value)

In [79]:
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.sort_values(by='Total_patients', ascending = False)
tk_2010_2015_code1_ct_large = tk_2010_2015_code1_ct.iloc[0:3]
tk_2010_2015_code1_ct_large

,Month,Total_patients
11,12,267449
3,4,266187
10,11,256686


In [80]:
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct.iloc[3:12]
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct_ect['Total_patients'].sum()
tk_2010_2015_code1_ct_large.loc[4] = ['ect', tk_2010_2015_code1_ct_ect]
tk_2010_2015_code1_ct_graph = tk_2010_2015_code1_ct_large.reset_index().drop('index', axis = 1)
tk_2010_2015_code1_ct_graph

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Month,Total_patients
0,12,267449
1,4,266187
2,11,256686
3,ect,1970021


In [81]:
color = ['forestgreen', 'limegreen', 'palegreen', 'honeydew']
tk_2010_2015_code1_ct_graph.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 90, labels = tk_2010_2015_code1_ct_graph['Month'], legend = True, colors = color)
# plt.pie(tk_2010_2015_code1_ct_graph['Total_patients'], shadow = True, startangle = 0, autopct='%1.1f%%')
plt.axis('equal')
plt.tight_layout()
savefig('이비인후과 월별 이용 인원 pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find 3 large main_sick with Month

In [82]:
ms_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind', 'main_sick']]
ms_2010_2015['rec_date'] = pd.to_datetime(ms_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
ms_2010_2015_code1 = ms_2010_2015[ms_2010_2015['treat_kind'] == 13]
ms_2010_2015_code1_st = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[0, 'Month']
ms_2010_2015_code1_st = ms_2010_2015_code1[ms_2010_2015_code1_st]
ms_2010_2015_code1_st.head()

,rec_date,treat_kind,main_sick
6,2013-12-11,13,J340
7,2013-12-23,13,J039
8,2013-12-02,13,J209
9,2013-12-13,13,J209
10,2013-12-17,13,J209


In [83]:
ms_2010_2015_code1_nd = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[1, 'Month']
ms_2010_2015_code1_nd = ms_2010_2015_code1[ms_2010_2015_code1_nd]
ms_2010_2015_code1_nd.head()

,rec_date,treat_kind,main_sick
49379,2013-04-09,13,J039
49380,2013-04-09,13,J039
49381,2013-04-15,13,J039
49382,2013-04-26,13,J039
49383,2013-04-12,13,J039


In [84]:
ms_2010_2015_code1_th = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[2, 'Month']
ms_2010_2015_code1_th = ms_2010_2015_code1[ms_2010_2015_code1_th]
ms_2010_2015_code1_th.head()

,rec_date,treat_kind,main_sick
4322,2013-11-16,13,K120
4325,2013-11-01,13,J039
4344,2013-11-19,13,K120
6233,2013-11-21,13,J042
6234,2013-11-11,13,J310


## First Month Find 3 Large main_sick

In [85]:
a1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [86]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
#code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    107
 Name: A090, dtype: int64, counts    67
 Name: B002, dtype: int64, counts    324
 Name: C73, dtype: int64, counts    86
 Name: D34, dtype: int64, counts    104
 Name: E041, dtype: int64, counts    17
 Name: F458, dtype: int64, counts    140
 Name: G510, dtype: int64, counts    6955
 Name: H660, dtype: int64, counts    126
 Name: I109, dtype: int64, counts    34675
 Name: J209, dtype: int64, counts    1222
 Name: K219, dtype: int64, counts    1388
 Name: L040, dtype: int64, counts    418
 Name: M791, dtype: int64, counts    23
 Name: N400, dtype: int64, counts    5
 Name: P220, dtype: int64, counts    149
 Name: Q181, dtype: int64, counts    1107
 Name: R040, dtype: int64, counts    170
 Name: S022, dtype: int64, counts    475
 Name: T16, dtype: int64, counts    26
 Name: Z011, dtype: int64]

In [90]:
code1_st_graph1 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H660')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'H660': 'Acute suppurative otitis media'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L040')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'L040': 'Acute lymphadenitis of face, head and neck'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'December', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('이비인후과 12월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Second Month Find 3 Large main_sick

In [88]:
a1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [89]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
#code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
#code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    88
 Name: A090, dtype: int64, counts    87
 Name: B370, dtype: int64, counts    318
 Name: C73, dtype: int64, counts    76
 Name: D110, dtype: int64, counts    84
 Name: E041, dtype: int64, counts    17
 Name: F510, dtype: int64, counts    154
 Name: G473, dtype: int64, counts    7321
 Name: H660, dtype: int64, counts    92
 Name: I109, dtype: int64, counts    31305
 Name: J209, dtype: int64, counts    1303
 Name: K219, dtype: int64, counts    1322
 Name: L040, dtype: int64, counts    362
 Name: M791, dtype: int64, counts    23
 Name: N400, dtype: int64, counts    110
 Name: Q181, dtype: int64, counts    1199
 Name: R040, dtype: int64, counts    165
 Name: S092, dtype: int64, counts    447
 Name: T16, dtype: int64, counts    29
 Name: Z011, dtype: int64]

In [91]:
code1_st_graph1 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H660')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'H660': 'Acute suppurative otitis media'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('L040')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'L040': 'Acute lymphadenitis of face, head and neck'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'April', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('이비인후과 4월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Third Month Find 3 Large main_sick

In [92]:
a1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [93]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
#code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
#code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
#code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    68
 Name: A090, dtype: int64, counts    72
 Name: B002, dtype: int64, counts    332
 Name: C73, dtype: int64, counts    79
 Name: D34, dtype: int64, counts    86
 Name: E041, dtype: int64, counts    15
 Name: F458, dtype: int64, counts    146
 Name: G473, dtype: int64, counts    6047
 Name: H660, dtype: int64, counts    118
 Name: I109, dtype: int64, counts    32714
 Name: J209, dtype: int64, counts    1185
 Name: K219, dtype: int64, counts    1304
 Name: L040, dtype: int64, counts    354
 Name: M791, dtype: int64, counts    144
 Name: Q181, dtype: int64, counts    931
 Name: R040, dtype: int64, counts    161
 Name: S022, dtype: int64, counts    471
 Name: T16, dtype: int64, counts    20
 Name: Z011, dtype: int64]

In [94]:
code1_st_graph1 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J209')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'J209': 'Acute bronchitis, unspecified'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H660')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'H660': 'Acute suppurative otitis media'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('L040')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'L040': 'Acute lymphadenitis of face, head and neck'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'November', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('이비인후과 11월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Counting Patient with treat_kind 14 (Month Criteria)

In [95]:
tk_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind']]
tk_2010_2015['rec_date'] = pd.to_datetime(tk_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
tk_2010_2015_code1 = tk_2010_2015[tk_2010_2015['treat_kind'] == 12]
tk_2010_2015_code1_ct = tk_2010_2015_code1['rec_date'].groupby(tk_2010_2015_code1.rec_date.dt.month).agg('count')
tk_2010_2015_code1_ct = pd.DataFrame(tk_2010_2015_code1_ct)
plt.figure()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns= {'rec_date': 'Total_patients'})
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.reset_index()
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.rename(columns = {'rec_date': 'Month'})
tk_2010_2015_code1_ct

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


,Month,Total_patients
0,1,166297
1,2,148583
2,3,170100
3,4,176023
4,5,177333
5,6,157508
6,7,175058
7,8,179992
8,9,164270
9,10,161834


## Visualization treat_kind 14 with Month

In [96]:
f, ax = plt.subplots(figsize = (10, 15))

# plot the total treat_kind 14
sns.barplot(x = 'Month', y = 'Total_patients', data = tk_2010_2015_code1_ct)
plt.savefig('안과 월별 이용 인원.png', dpi=1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find max value and making new table (3 large value)

In [97]:
tk_2010_2015_code1_ct = tk_2010_2015_code1_ct.sort_values(by='Total_patients', ascending = False)
tk_2010_2015_code1_ct_large = tk_2010_2015_code1_ct.iloc[0:3]
tk_2010_2015_code1_ct_large

,Month,Total_patients
7,8,179992
4,5,177333
3,4,176023


In [98]:
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct.iloc[3:12]
tk_2010_2015_code1_ct_ect = tk_2010_2015_code1_ct_ect['Total_patients'].sum()
tk_2010_2015_code1_ct_large.loc[1] = ['ect', tk_2010_2015_code1_ct_ect]
tk_2010_2015_code1_ct_graph = tk_2010_2015_code1_ct_large.reset_index().drop('index', axis = 1)
tk_2010_2015_code1_ct_graph

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Month,Total_patients
0,8,179992
1,5,177333
2,4,176023
3,ect,1455397


In [99]:
color = ['forestgreen', 'limegreen', 'palegreen', 'honeydew']
tk_2010_2015_code1_ct_graph.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 90, labels = tk_2010_2015_code1_ct_graph['Month'], legend = True, colors = color)
# plt.pie(tk_2010_2015_code1_ct_graph['Total_patients'], shadow = True, startangle = 0, autopct='%1.1f%%')
plt.axis('equal')
plt.tight_layout()
savefig('안과 월별 이용 인원 pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## Find 3 large main_sick with Month

In [100]:
ms_2010_2015 = JR_2010_2015[['rec_date', 'treat_kind', 'main_sick']]
ms_2010_2015['rec_date'] = pd.to_datetime(ms_2010_2015['rec_date'], format = '%Y%m%d', errors = 'coerce')
ms_2010_2015_code1 = ms_2010_2015[ms_2010_2015['treat_kind'] == 12]
ms_2010_2015_code1_st = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[0, 'Month']
ms_2010_2015_code1_st = ms_2010_2015_code1[ms_2010_2015_code1_st]
ms_2010_2015_code1_st.head()

,rec_date,treat_kind,main_sick
90711,2013-08-01,12,H258
93811,2013-08-07,12,H251
93812,2013-08-08,12,H259
93846,2013-08-02,12,H260
93847,2013-08-09,12,H250


In [101]:
ms_2010_2015_code1_nd = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[1, 'Month']
ms_2010_2015_code1_nd = ms_2010_2015_code1[ms_2010_2015_code1_nd]
ms_2010_2015_code1_nd.head()

,rec_date,treat_kind,main_sick
1240,2013-05-13,12,H258
7894,2013-05-14,12,H258
55793,2013-05-03,12,H251
55799,2013-05-03,12,H259
57068,2013-05-03,12,H251


In [102]:
ms_2010_2015_code1_th = ms_2010_2015_code1.rec_date.dt.month == tk_2010_2015_code1_ct_graph.ix[2, 'Month']
ms_2010_2015_code1_th = ms_2010_2015_code1[ms_2010_2015_code1_th]
ms_2010_2015_code1_th.head()

,rec_date,treat_kind,main_sick
22365,2013-04-25,12,H258
30375,2013-04-05,12,H251
45631,2013-04-01,12,H258
45632,2013-04-01,12,H258
45650,2013-04-01,12,H258


## First Month Find 3 Large main_sick

In [103]:
a1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [104]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
#code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    1
 Name: A154, dtype: int64, counts    2093
 Name: B300, dtype: int64, counts    5
 Name: C696, dtype: int64, counts    46
 Name: D310, dtype: int64, counts    131
 Name: E143, dtype: int64, counts    1
 Name: F03, dtype: int64, counts    20
 Name: G245, dtype: int64, counts    15553
 Name: H101, dtype: int64, counts    23
 Name: I109, dtype: int64, counts    45
 Name: J303, dtype: int64, counts    66
 Name: K30, dtype: int64, counts    33
 Name: L039, dtype: int64, counts    56
 Name: M350, dtype: int64, counts    6
 Name: N185, dtype: int64, counts    1
 Name: O141, dtype: int64, counts    66
 Name: Q103, dtype: int64, counts    4
 Name: R42, dtype: int64, counts    467
 Name: S050, dtype: int64, counts    1167
 Name: T151, dtype: int64, counts    1868
 Name: Z961, dtype: int64]

In [105]:
code1_st_graph1 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('H101')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'H101': 'Acute atopic conjunctivitis'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('B300')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'B300': 'Keratoconjunctivitis due to adenovirus'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_st[ms_2010_2015_code1_st['main_sick'].str.match('Z961')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'Z961': 'Presence of intraocular lens'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'August', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('안과 8월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Second Month Find 3 Large main_sick

In [106]:
a1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [107]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
#code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    1
 Name: A165, dtype: int64, counts    1340
 Name: B300, dtype: int64, counts    9
 Name: C692, dtype: int64, counts    56
 Name: D310, dtype: int64, counts    136
 Name: E143, dtype: int64, counts    1
 Name: F950, dtype: int64, counts    30
 Name: G245, dtype: int64, counts    15452
 Name: H041, dtype: int64, counts    24
 Name: I109, dtype: int64, counts    48
 Name: J303, dtype: int64, counts    90
 Name: K30, dtype: int64, counts    22
 Name: L020, dtype: int64, counts    48
 Name: M350, dtype: int64, counts    2
 Name: N185, dtype: int64, counts    8
 Name: P391, dtype: int64, counts    55
 Name: Q103, dtype: int64, counts    13
 Name: R12, dtype: int64, counts    543
 Name: S050, dtype: int64, counts    1227
 Name: T151, dtype: int64, counts    2610
 Name: Z961, dtype: int64]

In [109]:
code1_st_graph1 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('H041')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'H041': 'Other disorders of lacrimal gland'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('Z961')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'Z961': 'Presence of intraocular lens'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_nd[ms_2010_2015_code1_nd['main_sick'].str.match('B300')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'B300': 'Keratoconjunctivitis due to adenovirus'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'May', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('안과 5월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

## Third Month Find 3 Large main_sick¶

In [110]:
a1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('A')]['main_sick'].values, sort = True)
b1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('B')]['main_sick'].values, sort = True)
c1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('C')]['main_sick'].values, sort = True)
d1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('D')]['main_sick'].values, sort = True)
e1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('E')]['main_sick'].values, sort = True)
f1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('F')]['main_sick'].values, sort = True)
g1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('G')]['main_sick'].values, sort = True)
h1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H')]['main_sick'].values, sort = True)
i1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('I')]['main_sick'].values, sort = True)
j1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('J')]['main_sick'].values, sort = True)
k1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('K')]['main_sick'].values, sort = True)
l1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('L')]['main_sick'].values, sort = True)
m1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('M')]['main_sick'].values, sort = True)
n1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('N')]['main_sick'].values, sort = True)
o1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('O')]['main_sick'].values, sort = True)
p1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('P')]['main_sick'].values, sort = True)
q1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Q')]['main_sick'].values, sort = True)
r1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('R')]['main_sick'].values, sort = True)
s1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('S')]['main_sick'].values, sort = True)
t1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('T')]['main_sick'].values, sort = True)
u1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('U')]['main_sick'].values, sort = True)
v1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('V')]['main_sick'].values, sort = True)
w1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('W')]['main_sick'].values, sort = True)
x1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('X')]['main_sick'].values, sort = True)
y1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Y')]['main_sick'].values, sort = True)
z1 = pd.value_counts(ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Z')]['main_sick'].values, sort = True)

In [111]:
code1_list = []
a1 = pd.DataFrame(a1)
a1 = a1.rename(columns = {0: 'counts'})
a1_max = a1.iloc[a1['counts'].values.argmax()]
code1_list.append(a1_max)
b1 = pd.DataFrame(b1)
b1 = b1.rename(columns = {0: 'counts'})
code1_list.append(b1.iloc[b1['counts'].values.argmax()])
c1 = pd.DataFrame(c1)
c1 = c1.rename(columns = {0: 'counts'})
code1_list.append(c1.iloc[c1['counts'].values.argmax()])
d1 = pd.DataFrame(d1)
d1 = d1.rename(columns = {0: 'counts'})
code1_list.append(d1.iloc[d1['counts'].values.argmax()])
e1 = pd.DataFrame(e1)
e1 = e1.rename(columns = {0: 'counts'})
code1_list.append(e1.iloc[e1['counts'].values.argmax()])
f1 = pd.DataFrame(f1)
f1 = f1.rename(columns = {0: 'counts'})
code1_list.append(f1.iloc[f1['counts'].values.argmax()])
g1 = pd.DataFrame(g1)
g1 = g1.rename(columns = {0: 'counts'})
code1_list.append(g1.iloc[g1['counts'].values.argmax()])
h1 = pd.DataFrame(h1)
h1 = h1.rename(columns = {0: 'counts'})
code1_list.append(h1.iloc[h1['counts'].values.argmax()])
i1 = pd.DataFrame(i1)
i1 = i1.rename(columns = {0: 'counts'})
code1_list.append(i1.iloc[i1['counts'].values.argmax()])
j1 = pd.DataFrame(j1)
j1 = j1.rename(columns = {0: 'counts'})
code1_list.append(j1.iloc[j1['counts'].values.argmax()])
k1 = pd.DataFrame(k1)
k1 = k1.rename(columns = {0: 'counts'})
code1_list.append(k1.iloc[k1['counts'].values.argmax()])
l1 = pd.DataFrame(l1)
l1 = l1.rename(columns = {0: 'counts'})
code1_list.append(l1.iloc[l1['counts'].values.argmax()])
m1 = pd.DataFrame(m1)
m1 = m1.rename(columns = {0: 'counts'})
code1_list.append(m1.iloc[m1['counts'].values.argmax()])
n1 = pd.DataFrame(n1)
n1 = n1.rename(columns = {0: 'counts'})
code1_list.append(n1.iloc[n1['counts'].values.argmax()])
o1 = pd.DataFrame(o1)
o1 = o1.rename(columns = {0: 'counts'})
#code1_list.append(o1.iloc[o1['counts'].values.argmax()])
p1 = pd.DataFrame(p1)
p1 = p1.rename(columns = {0: 'counts'})
#code1_list.append(p1.iloc[p1['counts'].values.argmax()])
q1 = pd.DataFrame(q1)
q1 = q1.rename(columns = {0: 'counts'})
code1_list.append(q1.iloc[q1['counts'].values.argmax()])
r1 = pd.DataFrame(r1)
r1 = r1.rename(columns = {0: 'counts'})
code1_list.append(r1.iloc[r1['counts'].values.argmax()])
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns = {0: 'counts'})
code1_list.append(s1.iloc[s1['counts'].values.argmax()])
t1 = pd.DataFrame(t1)
t1 = t1.rename(columns = {0: 'counts'})
code1_list.append(t1.iloc[t1['counts'].values.argmax()])
u1 = pd.DataFrame(u1)
u1 = u1.rename(columns = {0: 'counts'})
#code1_list.append(u1.iloc[u1['counts'].values.argmax()])
v1 = pd.DataFrame(v1)
v1 = v1.rename(columns = {0: 'counts'})
#code1_list.append(v1.iloc[v1['counts'].values.argmax()])
w1 = pd.DataFrame(w1)
w1 = w1.rename(columns = {0: 'counts'})
#code1_list.append(w1.iloc[w1['counts'].values.argmax()])
x1 = pd.DataFrame(x1)
x1 = x1.rename(columns = {0: 'counts'})
#code1_list.append(x1.iloc[x1['counts'].values.argmax()])
y1 = pd.DataFrame(y1)
y1 = y1.rename(columns = {0: 'counts'})
#code1_list.append(y1.iloc[y1['counts'].values.argmax()])
z1 = pd.DataFrame(z1)
z1 = z1.rename(columns = {0: 'counts'})
code1_list.append(z1.iloc[z1['counts'].values.argmax()])
code1_list

[counts    2
 Name: A151, dtype: int64, counts    1229
 Name: B300, dtype: int64, counts    9
 Name: C439, dtype: int64, counts    52
 Name: D310, dtype: int64, counts    158
 Name: E143, dtype: int64, counts    1
 Name: F950, dtype: int64, counts    33
 Name: G245, dtype: int64, counts    15724
 Name: H041, dtype: int64, counts    31
 Name: I109, dtype: int64, counts    44
 Name: J039, dtype: int64, counts    104
 Name: K30, dtype: int64, counts    29
 Name: L020, dtype: int64, counts    51
 Name: M350, dtype: int64, counts    6
 Name: N185, dtype: int64, counts    64
 Name: Q103, dtype: int64, counts    15
 Name: R42, dtype: int64, counts    573
 Name: S050, dtype: int64, counts    1249
 Name: T151, dtype: int64, counts    2902
 Name: Z961, dtype: int64]

In [112]:
code1_st_graph1 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('H041')]
code1_st_graph1['main_sick'] = code1_st_graph1['main_sick'].map({'H041': 'Other disorders of lacrimal gland'})
code1_st_graph1 = pd.value_counts(code1_st_graph1['main_sick'].values, sort = False)
code1_st_graph1 = pd.DataFrame(code1_st_graph1)
code1_st_graph2 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('Z961')]
code1_st_graph2['main_sick'] = code1_st_graph2['main_sick'].map({'Z961': 'Presence of intraocular lens'})
code1_st_graph2 = pd.value_counts(code1_st_graph2['main_sick'].values, sort = False)
code1_st_graph2 = pd.DataFrame(code1_st_graph2)
code1_st_graph3 = ms_2010_2015_code1_th[ms_2010_2015_code1_th['main_sick'].str.match('T151')]
code1_st_graph3['main_sick'] = code1_st_graph3['main_sick'].map({'T151': 'Foreign body in conjunctival sac'})
code1_st_graph3 = pd.value_counts(code1_st_graph3['main_sick'].values, sort = False)
code1_st_graph3 = pd.DataFrame(code1_st_graph3)
c1_st_123 = pd.concat([code1_st_graph1, code1_st_graph2, code1_st_graph3]).reset_index()
c1_st_123 = c1_st_123.rename(columns = {'index' : 'main_sick', 0: 'Total_patients'})
color = ['limegreen', 'palegreen', 'honeydew']
c1_st_123.plot(kind = 'pie', y = 'Total_patients', autopct = '%1.1f%%', startangle = 180, labels = c1_st_123['main_sick'], legend = True, title = 'April', colors = color, figsize = (15, 10))
plt.axis('equal')
plt.tight_layout()
savefig('안과 4월 main_sick pie chart.png', dpi = 1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/